In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import functools
import scipy.signal
import tsfresh

from tqdm import tqdm

In [ ]:
%%time

filepath = '/home/sergey/Projects/Kaggle/LANL-Earthquake-Prediction/train.csv'

df = pd.read_csv(filepath, 
                 dtype={"acoustic_data": np.int16, "time_to_failure": np.float32}, 
                 skiprows=1, 
                 names=['s', 'ttf'])

In [ ]:
filepath = '/home/sergey/Projects/Kaggle/LANL-Earthquake-Prediction/train/train.h5'

df.to_hdf(filepath, key='table')

In [2]:
filepath = '/home/sergey/Projects/Kaggle/LANL-Earthquake-Prediction/train/train.h5'

df = pd.read_hdf(filepath, key='table')

In [3]:
def window_decorator(window_size=10000):
    def window_calc(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            temp = []
            df = args[0]
            array_size = df.shape[0]
            for i in tqdm(range(0, array_size, window_size)):
                batch = df.iloc[i: i+window_size].values
                temp.append(func(batch, *args, **kwargs))
            return pd.DataFrame(temp, columns={func.__name__})
        return wrapper
    return window_calc

In [4]:
@window_decorator()
def w_psd(df, *args, **kwargs):
    return np.sum(scipy.signal.periodogram(df, kwargs['fs'])[1])

@window_decorator()
def w_mean(df, *args, **kwargs):
    return np.mean(df)

@window_decorator()
def w_std(df, *args, **kwargs):
    return np.std(df)

@window_decorator()
def w_min(df, *args, **kwargs):
    return np.min(df)

@window_decorator()
def w_max(df, *args, **kwargs):
    return np.max(df)

@window_decorator()
def w_min(df, *args, **kwargs):
    return np.min(df)

@window_decorator()
def w_abs_energy(df, *args, **kwargs):
    return tsfresh.feature_extraction.feature_calculators.abs_energy(df)

@window_decorator()
def w_binned_entropy(df, *args, **kwargs):
    return tsfresh.feature_extraction.feature_calculators.binned_entropy(df, kwargs['max_bins'])

1 2 3


In [ ]:
def get_df_train(df, func_list):
    
    return pd.concat([func(df['s']) for func in func_list], axis=1)

In [14]:


df_temp = pd.concat([w_mean(df['s']), w_min(df['s']), w_psd(df['s'])], axis=1)
df_temp

100%|██████████| 62915/62915 [00:04<00:00, 13693.71it/s]


,w_mean,w_min
0,5.182000,-98
1,4.917400,-23
2,4.935700,-16
3,4.961500,-24
4,4.813900,-35
5,4.764700,-19
6,5.152700,-56
7,5.182300,-28
8,5.061300,-37
9,5.187900,-32


In [ ]:
w_abs_energy(df['s'])

In [ ]:
w_psd(df['s'])

In [ ]:
w_std(df['s'])

In [25]:
from inspect import getmembers, isfunction

functions_list = [o for o in getmembers(tsfresh.feature_extraction.feature_calculators) if isfunction(o[1])]
t = functions_list[4][1]

14